<a href="https://colab.research.google.com/github/granantuin/Model_vs_data_label/blob/master/tensorflow_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
from sklearn.metrics import confusion_matrix ,accuracy_score,f1_score,recall_score
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)


In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
y_data=pd.read_excel("/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_Mars_H24to48_dir/y_data.xlsx")
x_data=pd.read_csv("/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_Mars_H24to48_dir/x_data",usecols=range(1,10))

In [0]:
NE=y_data[(y_data["value"]>22.5) & (y_data["value"]<67.5)]
NE["label"]="NE"
E=y_data[(y_data["value"]>67.5) & (y_data["value"]<112.5)]
E["label"]="E"
SE=y_data[(y_data["value"]>112.5) & (y_data["value"]<157.5)]
SE["label"]="SE"
S=y_data[(y_data["value"]>157.5) & (y_data["value"]<202.5)]
S["label"]="S"
SW=y_data[(y_data["value"]>202.5) & (y_data["value"]<247.5)]
SW["label"]="SW"
W=y_data[(y_data["value"]>247.5) & (y_data["value"]<292.5)]
W["label"]="W"
NW=y_data[(y_data["value"]>292.5) & (y_data["value"]<337.5)]
NW["label"]="NW"
N=y_data[(y_data["value"]>337.5) | (y_data["value"]<22.5)]
N["label"]="N"
winds=[NE,N,E,SE,S,SW,W,NW]
y_data=pd.concat(winds)
y_data=y_data.sort_values(by="DATA (Horario UTC)")


In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.4, random_state=5)

In [0]:
l0 = tf.keras.layers.Dense(units=5, input_shape=[9]) 
l1 = tf.keras.layers.Dense(units=5,)
l2 = tf.keras.layers.Dense(units=1,)
model = tf.keras.Sequential([l0,l1,l2])

In [0]:
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(0.1))

In [0]:
print("These are the layer variables: {}".format(l0.get_weights()))

In [0]:
history = model.fit(x_train, y_train, epochs=500, verbose=False)
print("Finished training the model")

In [0]:
plt.xlabel('Epoch Number')
plt.ylabel("Loss Magnitude")
plt.plot(history.history['loss'])

In [0]:
y_pred=model.predict(x_test)
y_pred=y_pred.reshape(1,-1)[0]

In [0]:
index=["E","N","NE","NW","S","SE","SW","W"]
print(pd.DataFrame(confusion_matrix(y_test, y_pred), index=index, columns=index))
print("****************")
print("Accuracy=","{:.2%}".format(accuracy_score(y_test, y_pred)))
print("****************")
results= precision_recall_fscore_support(y_test, y_pred, average=None, )
df=pd.DataFrame({"Precision":results[0],"Recall":results[1],"F1":results[2],"W_DIR":index})
df=df.set_index("W_DIR")
print("Average precision =","{:.2%}".format(df["Precision"].mean()))
print("Average recall =","{:.2%}".format(df["Recall"].mean()))
print(df)